In [9]:
import pandas as pd

In [10]:
file_path = ".\某医院2018年药品销售数据.xlsx"
df = pd.read_excel(file_path)

## 1. 查看数据

In [11]:
# 查看前五条数据
df.head()

,购药时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01 星期五,1.616528e+06,236701.0,强力VC银翘片,6.0,82.8,69.00
1,2018-01-02 星期六,1.616528e+06,236701.0,清热解毒口服液,1.0,28.0,24.64
2,2018-01-06 星期三,1.260283e+07,236701.0,感康,2.0,16.8,15.00
3,2018-01-11 星期一,1.007034e+10,236701.0,三九感冒灵,1.0,28.0,28.00
4,2018-01-15 星期五,1.015543e+08,236701.0,三九感冒灵,8.0,224.0,208.00


In [12]:
# 查看数据量
df.shape

(6578, 7)

In [13]:
# 表结构
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6578 entries, 0 to 6577
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   购药时间    6576 non-null   object 
 1   社保卡号    6576 non-null   float64
 2   商品编码    6577 non-null   float64
 3   商品名称    6577 non-null   object 
 4   销售数量    6577 non-null   float64
 5   应收金额    6577 non-null   float64
 6   实收金额    6577 non-null   float64
dtypes: float64(5), object(2)
memory usage: 359.9+ KB


In [14]:
# 查看缺失值数量
pd.DataFrame(df.isnull().sum(), columns=["缺失值数量"])

,缺失值数量
购药时间,2
社保卡号,2
商品编码,1
商品名称,1
销售数量,1
应收金额,1
实收金额,1


## 2. 数据清洗

In [15]:
# 缺失值很少，直接删除
df.dropna(how="any", inplace=True)
print("删除后的数据量：",df.shape)
pd.DataFrame(df.isnull().sum(), columns=["缺失值数量"])

删除后的数据量： (6575, 7)


,缺失值数量
购药时间,0
社保卡号,0
商品编码,0
商品名称,0
销售数量,0
应收金额,0
实收金额,0


In [16]:
# 查找异常值
df[df["销售数量"] <= 0].head() # 销售数量为0或者为负数

,购药时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
61,2018-05-02 星期一,1.161653e+07,236702.0,强力VC银翘片,-1.0,-13.8,-13.8
100,2018-06-19 星期日,1.148763e+07,236704.0,氨加黄敏胶囊,-1.0,-12.4,-11.0
201,2018-02-24 星期三,1.001256e+10,236706.0,阿司匹林,-2.0,-93.6,-84.0
271,2018-01-31 星期日,1.616528e+06,236709.0,心痛定,-1.0,-44.8,-39.8
439,2018-01-17 星期日,1.263213e+07,2367011.0,开博通,0.0,0.0,0.0


In [17]:
# 删除异常值
df.drop(df[df["销售数量"] <= 0].index, inplace=True)
# 查看数据量
df.shape

(6532, 7)

In [18]:
# 把购药时间分成日期和星期两个字段
df[["日期","星期"]] = df["购药时间"].str.split(" ",expand=True)

# 删除原来的"购药时间"和"社保卡号"
df.drop("购药时间", axis=1)

,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额,日期,星期
0,1.616528e+06,236701.0,强力VC银翘片,6.0,82.8,69.00,2018-01-01,星期五
1,1.616528e+06,236701.0,清热解毒口服液,1.0,28.0,24.64,2018-01-02,星期六
2,1.260283e+07,236701.0,感康,2.0,16.8,15.00,2018-01-06,星期三
3,1.007034e+10,236701.0,三九感冒灵,1.0,28.0,28.00,2018-01-11,星期一
4,1.015543e+08,236701.0,三九感冒灵,8.0,224.0,208.00,2018-01-15,星期五
...,...,...,...,...,...,...,...,...
6572,1.006048e+10,2367011.0,高特灵,1.0,5.6,5.00,2018-04-27,星期三
6573,1.078861e+08,2367011.0,高特灵,10.0,56.0,54.80,2018-04-27,星期三
6575,1.008787e+10,2367011.0,高特灵,2.0,11.2,9.86,2018-04-27,星期三
6576,1.340663e+07,2367011.0,高特灵,1.0,5.6,5.00,2018-04-27,星期三


In [19]:
# 转换日期格式，格式有问题的转换为naT格式
df["日期"]= pd.to_datetime(df["日期"],format='%Y-%m-%d',errors='coerce')
# 删除转换后naT空值
df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6509 entries, 0 to 6577
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   购药时间    6509 non-null   object        
 1   社保卡号    6509 non-null   float64       
 2   商品编码    6509 non-null   float64       
 3   商品名称    6509 non-null   object        
 4   销售数量    6509 non-null   float64       
 5   应收金额    6509 non-null   float64       
 6   实收金额    6509 non-null   float64       
 7   日期      6509 non-null   datetime64[ns]
 8   星期      6509 non-null   object        
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 508.5+ KB


In [26]:
# 描述性分析
print(df.describe())

               社保卡号          商品编码         销售数量         应收金额         实收金额
count  6.509000e+03  6.509000e+03  6509.000000  6509.000000  6509.000000
mean   6.088969e+09  1.015717e+06     2.405285    50.908726    46.709935
std    4.889663e+09  5.123392e+05     2.364095    87.634645    80.983274
min    1.616528e+06  2.367010e+05     1.000000     1.200000     0.030000
25%    1.014234e+08  8.614560e+05     1.000000    14.000000    12.600000
50%    1.001650e+10  8.615070e+05     2.000000    28.000000    27.000000
75%    1.004859e+10  8.687840e+05     2.000000    59.600000    53.000000
max    1.283612e+10  2.367012e+06    50.000000  2950.000000  2650.000000


### 3. 数据分析

TypeError: __init__() got an unexpected keyword argument 'axis'